In [49]:
import math
import random
import time
import copy
import heapq
import operator
import datetime as dt
from IPython.core.display import HTML
with open('style.css', 'r') as file:
    css = file.read()
HTML(css)

# Kalah Rules

### Game Structure:
<p>The playing field consists of six houses per side and additionally one large house per player at the edge, the so-called storage. At the beginning of the game, four seeds are placed in each house except the storage.</p>

![title](images/Board.png)

### Gameplay:
<p>Alternately, a player selects a house on his side of the board. The player then places the seeds in the following houses in a counter-clockwise direction. If visited, a seed is placed in the own storage, the opponent's storage is left out. The player's turn ends if all seeds are placed.

There are some special rules, that apply if the last seed is placed in the player's storage or an empty player's house:
1. If the last seed is placed in the own storage, it is the player's turn again. 
2. If the last seed is placed in an empty house on the player's own side of the board, this seed and all the seeds in the opposite house are placed in the player's storage. This can only occur if the opposite house contains atleast one seed.</p>

### End of the game:
<p>The game ends when all the houses of one player have been emptied. The opposing player then places all remaining seed in his storage.</p>

### Object of the game:
<p>The winner is the player who has more seeds in his storage at the end of the game.</p>

Source: http://gambiter.com/mancala/Kalah.html (09.03.2022), http://www.iggamecenter.com/info/en/kalah.html (09.03.2022)

# Kalah game definition

### Basic definition:
<p>We define the game G as a six-tuple as follows so that a computer is able to play Kalah.</p>

$$G = \lt States, s0, Players, nextStates, finished, utility\gt$$

The components have the following meanings:

1. **States** is a set that contains all possible states of the game Kalah. A state of the game is represented by a list containing two lists, representing the houses of the two players. The first six values of each list represent the number of seeds in the player's houses represented by the letters **{A, B, C, D, E, F}**. The seventh and last value stands for the number of seeds in the corresponding player's storage. The start state of the game is for example:

$$[[4,4,4,4,4,4,0], [4,4,4,4,4,4,0]]$$

2. **s0 $\in$ States** is the start state.

3. **Players** is a list that contains the players. Kalah is a game for exactly two players, therefore this game's **Players** list only contains two elements.
4. **nextStates** is a function which calculates a set of states that are reachable by one move from Player p in the state s. To do so, the function receives a state **s $\in$ States** and a player **p $\in$ Players**. The signature is given as follows:

$$nextStates: States \times Players \rightarrow 2^{States}$$

5. **finished** is a function that takes a state **s $\in$ States** and checks if the game is finished, meaning that one of the players has emptied all their houses. The signature is: 
<br><br>
$$finished: States \rightarrow \mathbb{B}$$
<br>
The function finished is used to compute a set TerminalStates that contains all states from a finished game. This set is defined as follows: 
<br><br>
$$TerminalStates:= \{s \in States | finished(s)\}$$
<br>
6. **utility** is a function that calculates the value of the game for a player. Therefore, the function takes a state **s $\in$ TerminalStates** and a player **p $\in$ Players**. The value that the function returns is an element from the set **{-1, 0, 1}**. The player p has lost the game when the function returns -1. If the function returns 1, the player has won the game and if the value is 0, the game ends in a draw. The signature for the function is:

$$utility: TerminalStates \times Players \rightarrow \{-1, 0. 1\}$$

source: https://github.com/karlstroetmann/Artificial-Intelligence/blob/master/Lecture-Notes/artificial-intelligence.pdf, S.87, Abruf am 06.02.2022

# Project Structure

This notebook implements the core of the Kalah game project as it holds all relevant definitions and classes. 

First, the **global variables** are defined. These are used by several of the classes but not changed by them. The global variables contain the number of seeds which each player has at the beginning of the game as this number could vary from the standard value four to change the game difficulty. Other global variables are the start state of the game and a variable to help matching a state to the letter representation of the game board.

In the next section, the **global functions** are defined. These functions contain the basic game mechanics, for example how one move is made or which states can be reached from a specific game state. Additionally, they help calculating the current value of the game for one of the players and can determine if the game is finished. These global functions are not contained in the game class as they are not only used during the game, but also by the different AI player classes to analyze the game state and calculate their next moves. 

After all of the global definitions are done, the different classes are defined, starting with the different <tt>Player</tt> classes and finishing with the <tt>Game</tt> class.

All of the specific **<tt>Player</tt> classes** are subclasses from the class <tt>Player</tt>. With this implementation it is very easy to have any combination of player types in the game as they all function the same way. The core mechanic of each <tt>Player</tt> type is defined by the method `choose_house`. This method determines how this type of player chooses their next house when it is their turn. The <tt>Player</tt> classes included in this project are a <tt>Human Player</tt> class where a human decides based on a UI which house they choose next, and four different AI player classes to play against: <tt>Random_AI Player</tt>, <tt>Minimax Player</tt>, <tt>AlphaBeta Player</tt> and <tt>Scout Player</tt>. Additionally, there is a log system applied in the project with which games can be saved to a file and replayed. For this reason there is also a <tt>Repeated Player</tt> class.

The last class which is defined in this core notebook is the **<tt>Kalah_Game</tt> class**. An object of this class configures and runs one game. The configurations include the players which are given at initialization as well as a display mode which determines whether the status of the game should be printed to the console, visualized using the Visualization notebook or if it should not be shown at all, for example if two AIs compete against each other. In addition, the <tt>Kalah_Game</tt> class handles the file logging if required.

# Global Definitions

## Global Variables

Additionally to the classes, there are three global variables:
- `gSeedStartNumber`
- `gStartState`
- `gOrder`
- `gCounts`

The global variable `gSeedStartNumber` defines the number of seeds in every house at the start of the game. By the use of this variable it is possible to play the game in different difficulties. The standard value is four seeds per house.

The global variable `gStartState` generates the start state of the game from the variable `gSeedStartNumber`. As explained above, this results in the start state for the value four being:

$$[[4,4,4,4,4,4,0], [4,4,4,4,4,4,0]]$$

The global variable `gOrder` has the purpose to help converting the state of the game board, which is represented by a nested list, to letters. The letter representation is mainly for the UI, but also distributes to print and logging messages being easier to humanly comprehend.

The global variable `gCounts` is used to count the number of node inspections the different AI's perform.

In [2]:
gSeedStartNumber = 4

In [3]:
gStartState = [[gSeedStartNumber for _ in range(6)]+[0] for _ in range(2)]
gStartState

[[4, 4, 4, 4, 4, 4, 0], [4, 4, 4, 4, 4, 4, 0]]

In [4]:
gOrder = [['A','B','C','D','E','F','O'],
         ['a','b','c','d','e','f','o']]

In [5]:
gCounts = 0

#### Auxiliary functions for tuples and lists

The function `to_tuple` takes a list of the two lists that represent a game state. The function converts the list to a tuple of tuples and returns it.

The function `to_list` reverses that procedure.

In [6]:
def to_tuple(state_list):
    return tuple(tuple(s) for s in state_list)

In [7]:
def to_list(state_tuple):
    return list(list(s) for s in state_tuple)

## Global Game Functions

- `other_player`
- `move`
    - `place_seeds`
    - `check_another_turn`
    - `steal_if_possible`
- `next_states`
- `finished`
- `heuristic`
- `utility`
- `value`

### Function: other_player
The function *other_player(player_num)* returns the opponent's player number to a given player number. This means that it returns 1 for the input 0, and 0 for the input 1.

In [8]:
def other_player(player_num):
    return 1 - player_num 

### Function: setSeedNumber
The function `setSeedNumber(number)` is used to set the globale variables `gSeedStartNumber` and `gStartState` for the given number of starting seeds.


In [9]:
def setSeedNumber(number):
    global gSeedStartNumber
    gSeedStartNumber = number
    global gStartState
    gStartState = [[gSeedStartNumber for _ in range(6)]+[0] for _ in range(2)]

### Function move
The function `move` calculates the resulting state after a player chooses a house in their turn and whether they get another move.
Additionally, the function is used in `next_states` to calculate all possible following states from one state.

##### The calculations are implemented based on the following game rules:

1. The current player chooses one of his houses. The seeds from the chosen house are placed counterclockwise in the houses of both players and in the store of the current player. The store of the opponent is left out. Then, if no special rule applies, it is the turn of the opponent.

2. If the last seed is placed in the store of the current player, they get another move.

3. This rule applies if the last seed is placed in an empty house of the current player and the opposite house is not empty. If these conditions are met, the seed from this house and all seeds from the opposite house are placed in the store of the current player. Then it's the opponent's turn.

To implement the function `move`, several auxiliary functions are used:
- `place_seeds`
- `check_another_turn`
- `steal_if_possible`
<br>

#### Auxiliary Function: place_seeds
The auxiliary function `place_seeds` is used in the function `move` to place the seeds from the chosen house according to the first game rule.

To do so, the function takes the current game state, the seed number of the chosen player house and the number of that house (choice). While placing the seeds, the state of the game is updated with every seed. Finally, after having placed all seeds, the new state is returned together with the player and house numbers of the last house in which a seed has been placed.

In [10]:
def place_seeds(state, seeds, player_num, choice):
    placing_house_num = choice
    placing_player_num = player_num
    # Go through houses counterclockwise
    while seeds > 0:
        placing_house_num += 1
        # Skip opponent's store
        if placing_house_num == 6 and placing_player_num != player_num:
            continue
        # Switch to houses of the other player
        if placing_house_num > 6:
            placing_house_num = 0
            placing_player_num = other_player(placing_player_num)
        # Add seed to the currently visited house
        state[placing_player_num][placing_house_num] += 1
        seeds -= 1
    return state, placing_player_num, placing_house_num

#### Auxiliary Function: check_another_turn

The auxiliary function  `check_another_turn` checks if the current player should get another turn according to the second game rule. This is the case whenever the last seed of the player was placed in their own store.

The function receives the resulting game state after `place_seeds` was executed as well as the player and house numbers of the last house in which a seed has been placed. It returns True if the current player gets another turn and False otherwise.


In [11]:
def check_another_turn(state, player_num, last_player_num, last_house_num):
    return last_player_num == player_num and last_house_num == 6 and not finished(state)

#### Auxiliary Function: steal_if_possible
The auxiliary function `steal_if_possible` checks if the current player can steal any seeds from their opponent according to the third game rule. This is the case whenever the last seed of the player was placed in an own empty house and the opponent has any seeds in the opposite house. In this case the own last seed together with the seeds of the opposite house are placed in the current player's store.

The function receives the resulting game state after `place_seeds` was executed as well as the player and house numbers of the last house in which a seed has been placed. Finally, the (potentially) new state is returned.

In [12]:
def steal_if_possible(state, player_num, last_player_num, last_house_num):
    state = copy.deepcopy(state)
    if last_house_num == 6 or player_num != last_player_num:
        return state
    last_house_seeds = state[player_num][last_house_num]
    other_player_num = other_player(player_num)
    opponent_house_num = 5-last_house_num
    last_house_opponent_seeds = state[other_player_num][opponent_house_num]
    if last_house_seeds == 1 and last_house_opponent_seeds != 0:                
        # Collect all seeds to be rewarded and empty both houses
        received_seeds = last_house_seeds + last_house_opponent_seeds
        state[other_player_num][opponent_house_num] = 0
        state[player_num][last_house_num] = 0
        # Award all the seeds to the current player's store
        state[player_num][6] += received_seeds
    return state

#### Function move:

The function `move` uses all auxiliary functions described above to calculate the resulting game state from the current player's house choice. Therefore it is given the current game state, the number of the player who performs the move and the corresponding house number choice.

At first, the function creates a copy of the state so that no changes are directly applied to the state variable in the game class. Next, the number of seeds in the chosen house are saved and the house is emptied. Afterwards, `place_seeds` is used to place the seeds counterclockwise in the houses and `steal_if_possible` is called to perform any steals afterwards, if required. Finally, `another_turn` calculates if the current player should receive another turn and this information is returned together with the resulting new game state.

In [13]:
def move(state, player_num, choice):
    if choice not in range(6):
        raise ValueError("The choice must be 0, 1, 2, 3, 4 or 5!")
    new_state = copy.deepcopy(state)
    seeds = new_state[player_num][choice]
    new_state[player_num][choice] = 0
    # Place all seeds and check for special rules
    new_state, last_player_num, last_house_num = place_seeds(new_state, seeds, player_num, choice)
    new_state = steal_if_possible(new_state, player_num, last_player_num, last_house_num)
    another_turn = check_another_turn(new_state, player_num, last_player_num, last_house_num)
    return new_state, another_turn

### Function: next_states

The function `next_states` computes all reachable states resulting from the `state` and the player with the number `player_num`. A reachable state is a state, that can be reached during a single turn. We consider a turn to still be the same turn, even if the special rule for an extra turn applies.

To find the reachable states the function executes a `move` for each selectable house. A selectable house contains at least one seed. If the player gets another turn `next_states` is called recursively resulting state until the player does not get another turn. Afterwards all new states are saved in a list and returned. Additionally the choices required to reach each state are returned in the same list aswell.

In [14]:
def next_states(state, player_num):
    states = []
    for choice in range(6):
        # Check if choice is valid (at least one seed in house)
        if state[player_num][choice] == 0:
            continue   
        next_state, another_turn = move(state, player_num, choice)

        # Check if player has another turn
        # If so, next_states is called recursively until the player has no other turn 
        if another_turn:
            for s,choices in next_states(next_state, player_num):
                states.append((s,[choice] + choices))
        else:
            states.append((next_state,[choice]))

    return states

### Function: finished
The function `finished` checks if one of the players has no seeds in their house left and is therefore unable to take another turn. If this is the case, the function returns True, otherwise it returns False. 

In [15]:
def finished(state):
    for side in state:
        fin = True
        for house in side[:-1]:
            if house != 0:
                fin = False
        if fin:
            return True
    return False

### Function: heuristic

The function `heuristic` calculates a floating number between -1.0 and 1.0 based on the value of the current game state for the current player. Therefore it recieves the current player's `player_num` and the current `state`.

There are many possible solutions to defining a heuristic. In this case, the following function is defined as the heuristic of the game:

$$h_v = \dfrac{(playerStore - opponentStore)}{(totalSeedNum - playerStore - opponentStore)}$$

To improve the accuracy of this value and limit it to a maximum of 1 and a minimum of -1 we calculate the maximum possible value for the heuristic:

$$h_{max} = \dfrac{totalSeedNum-2}{2} $$

This is correct since with a higher `playerStore` the numerator rises and the denominator becomes lower. Furthermore the function is not called if there are no seeds on either player's side, hence 2 seeds have to be substracted to account for one seed being left on both sides respectively.
<br>Since $$h_{max} \ge 0 $$<br> we need to add 1 so the denominator can not become 0. Therefore this function is defined as:

$$h(state) = \dfrac{h_v}{h_{max}+1}$$

This heuristic is based on the number of seeds contained in the player's stores in relation to the seeds that are left on the board and therefore still available. By using this relation, the heuristic appreciates that the value of a single seed increases in the course of the game because there are less seeds left to obtain. 

An improved heuristic could also include the seeds that are in the player's houses or give special points for empty houses as they offer opportunities to steal seeds from the opponent. 
*This would make the heuristic significally more complex though.*

It is noteworthy, that the equation $$totalSeeds - playerStore - opponentStore$$

can never be 0 since the only possible scenario for that to happen is when no seeds are available on the board anymore. But if that happens this function never gets called because the function finished(state) holds true in this case.

In [16]:
def heuristic(state, player_num):
    playerStore = state[player_num][-1]
    opponentStore = state[other_player(player_num)][-1]
    totalSeedNum = 6*2*gSeedStartNumber
    maximum = (totalSeedNum - 2) / 2
    
    return ((playerStore-opponentStore)/(totalSeedNum-playerStore-opponentStore))/(maximum + 1)

### Function: utility
The  function `utility` receives the number of the current player `player_num` and uses the current `state` to calculate the utility of that `state` for that player. The recieved `state` is always part of the abstract set `TerminalStates`, that was defined in the definition of the game.

The function `utility` compares the total seeds from both players (their respective stores and the remaining seeds from their houses). If the current player has more seeds than the opponent, this function returns 1. If they have less seeds than the opponent, the function returns -1. If both players have an equal amount of seeds the game is considered a draw and the number 0 is returned.     

In [17]:
def utility(state, player_num):
    playerScore = sum(state[player_num])
    opponentScore = sum(state[other_player(player_num)])
    if playerScore > opponentScore:
        return 1
    elif playerScore == opponentScore:
        return 0
    else:
        return -1

### Function: value


#### Function: memoize

The function `memoize` computes a memoized version of a function `f` that is given as a parameter. At first a dictionary named `Cache` is defined that is used as a memory cache for the function `memoized`. Everytime the function `f` gets called, this function checks whether a call with the given arguments has occured at least once before. It does so by checking the dictionary `Cache` for the given parameters.

If the check holds true, then the assigned value in the dictionary is returned and the function `f` does not execute.

If the check fails, the function `f` gets executed and a new entry is added to the dictionary, containing the given parameters and the newly computed return value.

In [18]:
Cache = {}

def memoize(f):
    global Cache

    def f_memoized(*args):
        args = (to_tuple(args[0]),args[1],args[2])
        if args in Cache:
            return Cache[args]
        
        result = f(to_list(args[0]),args[1],args[2])
        Cache[args] = result
        return result

    return f_memoized

The recursive function `value` takes a `state` and a `player_num`. In addition, a `limit` is given to define the recursion depth. This function has two base cases:

1. If the given `state` is part of the abstract set `TerminalStates` and therefore a finished state. When this happens, the function  `utility` is called. 
2. If the given `limit` is 0 and therefore the maximum recursion limit has been reached. When this happens, the function `heuristic` gets called.

In both scenarios the global variable `gCounts` is increased by 1.

In every other case the function calls itself once for every state computed by the function `next_states`, whilst changing the `player_num` to the opponent's number. This continues until one of the base cases is reached.

In [19]:
@memoize
def value(state, player_num, limit):
    global gCounts

    if finished(state):
        gCounts += 1
        return utility(state, player_num)
    if limit == 0:
        gCounts += 1
        return heuristic(state, player_num)
    
    other = other_player(player_num)
    return max([-value(ns, other, limit-1) for ns,_ in next_states(state, player_num)])

# Player Class

#### Attributes:
- `player_type`
- `number`
- `name`

#### Methods:

- `__init__`
- `__str__`
- `_available_house`
- `choose_house`

The class <tt>Player</tt> is the abstract superclass that represents a Kalah game player. Since Kalah is a two-player zero-sum game, it consists of two players playing against each other.

The attribute `player_type` is only used in subclasses of the <tt>Player</tt> class. It contains a string representation of the name of the <tt>Player</tt> subclass. This makes it possible to print the specific type of a <tt>Player</tt> object. This is for example used in the process of creating log files. 

The attribute `number` of a player has either the value 0 or 1 depending on the order in which the players take their turns. 
If the `number` of the player is 0, they start the game by having the first turn.
Additionally, `number` represents the index of the `state` list which contains the player's list of house values.
The `number` is initially set to 0, to prevent faulty assignments of numbers during the initialization of the game. When the game gets initialized the second player automatically recieves the number 1.

The attribute `name` is a string that represents the player's name in the UI or print and logging messages.

### Method: __init__
The method `__init__` initializes the <tt>Player</tt> object and checks if the given `name` is a string. If one of these criteria is not met, an error message is raised.

### Method: __str__
The method `__str__` returns the `name` of the <tt>Player</tt> object for print messages.

In [20]:
class Player:
    def __init__(self, name):
        self.number = 0
        if isinstance(name, str):
            self.name = name
        else:
            raise ValueError("Name must be a string!")
            
    def __str__(self):
        return self.name

### Method: _available_houses

The private method `_available_houses` receives a list which represents the player's houses and their store. The method returns a list with the indices of all houses that contain at least one seed (list indices with a value higher than zero).

In [21]:
def _available_houses(self, own_houses):
    return [i for i, n in enumerate(own_houses) if n != 0]

Player._available_houses = _available_houses

### Method: choose_house

The abstract method `choose_house` receives the current state of the Kalah board and returns the index of the chosen house from the player's house list. This method is used to distinguish the player subclasses and is therefore implemented differently in each of them. There is no basic implementation of this method in the <tt>Player</tt> class.

In [22]:
def choose_house(self, current_state):
    pass

Player.choose_house = choose_house

## Human Player Class

The class <tt>Human</tt> inherits from <tt>Player</tt>.
It is used for humans playing against each other or against one of the AIs. 

### Method: __init__
The method `__init__` initializes the <tt>Human Player</tt> object calls the `__init__` function of the superclass `Player`. Additionally the variable `player_type` is set to identify the Object.


In [23]:
class Human(Player):
    
    def __init__(self, name):
        self.player_type = "Human"
        super().__init__(name)

### Method: choose_house
The method `choose_house` for the <tt>Human</tt> class is implemented as follows:

At first the own houses are extracted from the game state and the available house indices are calculated using the method `_available_houses`. Afterwards the human player is asked to choose one of the available houses via an input field. If the player enters an invalid value, they are asked to input another value until a valid value is given. The index of the chosen house is returned.

In [24]:
def choose_house(self, current_state):
    own_t,store = current_state[self.number][:6],current_state[self.number][6]
    available = self._available_houses(own_t)

    i_string = "Choose one of the available houses:\n"
    for i in available:
        i_string += f"{gOrder[self.number][i]}, "

    choice_str = ""
    letter_numbers = {gOrder[1][i]:i for i in range(6)}
    choice_str = input(i_string[:-2]+"\n").lower()

    while choice_str not in [k for k in letter_numbers if letter_numbers[k] in available]:
        print("Invalid Input!")
        choice_str = input(i_string[:-2]+"\n").lower()

    choice = letter_numbers[choice_str]
    IPython.display.clear_output()
    return choice
    
Human.choose_house = choose_house

## Repeated Player Class

The class <tt>Repeated_Player</tt> inherits from <tt>Player</tt>.
It is used for repeating the behavior of a player from a previous game on basis of a given log file.

### Method: __init__
The method `__init__` initializes the <tt>Repeated Player</tt> object and calls the `__init__` function of the superclass `Player`. Additionally the variable `player_type` is set to identify the Object.

Furthermore the functions recieves a list of moves `logged_moves`, which is used to later on to replay the game at a different time.

In [25]:
class Repeated_Player(Player):
    
    def __init__(self, name, logged_moves):
        self.player_type = "Repeated_Player"
        self.moves_to_repeat = logged_moves
        super().__init__(name)

### Method: choose_house
The method `choose_house` for the <tt>Repeated_Player</tt> class is implemented as follows:

The list `moves_to_repeat` is handled as a stack in this method. This means that the moves at the beginning of the list are extracted one by one. This process is repeated until the current state of the game matches the state of the last removed move from the `moves_to_repeat` list. The player's next move should now be at the beginning of the list.

In the next step, this move is also extracted from the list, but not removed as it could contain the current state for the next move of the player if they gain another turn. To make sure, that this move was actually done by this player in the logged game, the logged move player number is compared to the actual player number. If the numbers match, the choice from that move is returned.

In [26]:
def choose_house(self, current_state):
    
    state = self.moves_to_repeat.pop(0)[2]
    while state != current_state:
        if len(self.moves_to_repeat) == 0:
            raise ValueError("There is no move left to repeat!")
        state = self.moves_to_repeat.pop(0)[2]
        
    this_move = self.moves_to_repeat[0]
    if this_move[0] not in (-1,self.number):
        raise ValueError("This move was originally not played by this player!")
        
    choice = this_move[1]
        
    return choice

Repeated_Player.choose_house = choose_house

## Random_AI Player Class

The class <tt>Random_AI</tt> inherits from <tt>Player</tt>. It is a simple AI player implementation which chooses the houses at random.

### Method: __init__
The method `__init__` initializes the <tt>Random_AI Player</tt> object and calls the `__init__` function of the superclass `Player`. Additionally the variable `player_type` is set to identify the Object.

Furthermore this function recieves an Integer `seed` which is used to set the seed for the random number calculations provided by the `random` library.

In [27]:
import random as rn

class Random_AI(Player):
    
    def __init__(self, name, seed=0):
        self.player_type = "Random_AI"
        self.seed = seed
        rn.seed(seed)
        super().__init__(name)

### Method: choose_house
The method `choose_house` for the <tt>Random_AI</tt> class is implemented as follows:

At first, the own houses are extracted from the game state and the indices of the available houses are calculated using the method *_available_houses*. From this list, one is chosen at random using the function `choice` from the library `random`. Afterwards, this value is returned.

In [28]:
def choose_house(self, current_state):
    own_t,safe = current_state[self.number][:6],current_state[self.number][6]
    available = self._available_houses(own_t)
    choice = rn.choice(available)

    return choice

Random_AI.choose_house = choose_house

# Minimax Player Class

The class <tt>Minimax</tt> inherits from <tt>Player</tt>. It is an AI player implementation that chooses a following state by calculating all possible next states for a defined limit depth. It then chooses the option that is valued highest, by the function `value`. In the scenario the multiple options share the highest value, one of those options is picked at random.

The class <tt>Minimax</tt> has a modified `__init__` function that takes the additional argument `limit` which is only relevant to the method `choose_house`. There, the value of `limit` defines for how many recursion steps the function `value` should be called. This means, that `limit` defines how many next game states should be calculated to form the decision of the <tt>Minimax</tt> AI.

The `limit` should be chosen carefully, as higher limits increase the recursion depth linearly and the computing time exponentially. Additionally, `limit` has to be greater than 1 at all times. 

Ideally `limit` should be a number from 2 - 4 (incl.) for the <tt>Minimax</tt> AI.

In [29]:
class Minimax(Player):
    
    def __init__(self, name, limit, seed=0):
        self.player_type = "Minimax"
        self.limit = limit
        self.seed = seed
        rn.seed(seed)
        super().__init__(name)

### Method: choose_house

The method `choose_house` is the core method of the <tt>Minimax</tt> algorithm. It calculates the best choices for a <tt>Minimax</tt> player, using the previously defined functions *value(state, player_num, limit)* and *next_states(state, player_num)*.

First, the best possible value between -1 and 1 (-1 meaning loss, 1 meaning win) that can be reached with the current state of the game is calculated. Following that, all next possible states are checked and only those are elected as eligible, which reach the previously calculated best possible value.

Afterwards, the best choice is being made by comparing all eligible choices for the highest player's Kalah store value.

In [30]:
def choose_house(self, current_state):
    
    NS = next_states(current_state, self.number)
    bestVal = value(current_state, self.number,self.limit)

    BestChoices = [choices[0] for state,choices in NS if -value(state, other_player(self.number),self.limit-1) == bestVal]
    
    if len(BestChoices) == 0:
        raise ValueError(f'No choice for Minimax found! \nbestVal: {bestVal}\nBestChoices: {BestChoices}\nnext_states: {NS}')

    # Find best choice, where number of seeds in own Store is maximized (out of next_states)
    best_choice = rn.choice(BestChoices)

    return best_choice

Minimax.choose_house = choose_house

# AlphaBeta Player Class

The class <tt>AlphaBeta</tt> inherits from <tt>Player</tt>. It is an AI player implementation which chooses a following state by calculating all possible next states for a defined limit depth. It then chooses the option which guarantees them a higher number of seeds in their store than their opponent. If there are several options, the choice which results in the highest number of seeds in the player's store is chosen.

In [31]:
class AlphaBeta(Player):
    
    def __init__(self, name, limit, seed=0):
        self.player_type = "AlphaBeta"
        self.limit = limit
        self.seed = seed
        rn.seed(seed)
        super().__init__(name)

### AlphaBeta algorithm: 

AlphaBeta is an optimization technique for the Minimax algorithm. A disadvantage of the Minimax algorithm is that it checks all the nodes, which takes a long time. With the AlphaBeta algorithm the correct Minimax decision can be found with less node inspections due to the possibility of pruning branches of the game tree. The AphaBeta algorithm ignores nodes that don't have an impact on finding the best move.  
The algorithm takes two additional arguments $\alpha$ and $\beta$. These values are used as lower and upper boundaries for the value of a state. $\alpha$ is the highest value for the maximizing function that has been found, and $\beta$ is the lowest value for the minimizing function that has been found. When the condition $\beta$<=$\alpha$ is met, the adjacent nodes don't need to be evaluated.

The AlphaBeta algorithm consists of the functions *evaluate*, *maxValue* and *minValue* which are explained in the following section. The implementation of the AlphaBeta algorithm is depth limited because it would take far to long to calculate every game path to it's terminal state. It also uses progressive deepening and move ordering which are explained in more detail in the following.

### Function: evaluate
The function *evaluate* takes seven arguments:
- *state* is the state of the game,
- *player_num* is either 0 or 1 depending on whose perspective is being evaluated
- *playing_player* is either 0 or 1 depending on whose turn it currently is
- *limit* is an integer to define the recursion depth limit
- *f* is a function which is either the function *minValue* or *maxValue*
- *alpha* is the lower bound
- *beta* is the upper bound


The main purpose of evaluate is to implement a better version of Memoization for the Alpha-Beta-Search.

If *evaluate* recieves a state s, that is not already in the cache it calls the function f with all the other parameters left.The function *evaluate* uses a global variable gCache which is used to save the results of *evaluate*. The cache is implemented as a dictionary with the states, player_num, playing_player and the limit as keys. The stored values are pairs of a flag $\in$ {$\geq$, =, $\leq$} and a variable v which is the result of the function *evaluate*.  

If the tuple of the given state s, the player_num, the playing_player and the limit is already defined in gCache, the function *evaluate* uses the stored flag and value v as follows:

1. If the flag shows that the value is exact, v can be returned:<br><br>
    $$gCache[s] = ('=', v) \implies evaluate(s, player\_num, playing\_player, limit, f, \alpha, \beta) = v$$<br><br>

2. If the flag shows that the value is an upper bound the procedure works as follows:
    
    A. the value is less or equal than $\alpha$. The true value of the state is less or equal than $\alpha$ and can be returned:<br>
    <br>$$gCache[s] = ('\leq', v) \land v \leq \alpha \implies evaluate(s, player\_num, playing\_player, limit, f, \alpha, \beta) = v$$<br>
    
    B. the value is between $\alpha$ and $\beta$. The true value of the state is less or equal than v. The interval [$\alpha$,$\beta$] is transformed to [$\alpha$,v]:<br>
    <br>$$gCache[s] = ('\leq', v) \land \alpha < v < \beta \implies evaluate(s, player\_num, playing\_player, limit, f, \alpha, \beta) = f(s, player\_num, playing\_player, limit, \alpha, v)$$<br>
    
    C. the value is bigger or equal than $\beta$. The value doesn't help and so the the interval is left at [$\alpha$,$\beta$]:<br>
    <br>$$gCache[s] = ('\leq', v) \land v \geq \beta \implies evaluate(s, player\_num, playing\_player, limit, f, \alpha, \beta) = f(s, player\_num, playing\_player, limit, \alpha, \beta)$$<br>  
<br>

3. If the flag shows that the value is a lower bound the procedure works as follows:
    
    A. the value is greater or equal than $\beta$. The true value of the state is bigger or equal than $\beta$ and can be returned:<br>
    <br>$$gCache[s] = ('\geq', v) \land v \geq \beta \implies evaluate(s, player\_num, playing\_player, limit, f, \alpha, \beta) = v$$<br>
    
    B. the value is between $\alpha$ and $\beta$. The true value of the state is bigger or equal than v. The interval [$\alpha$,$\beta$] is transformed to [v,$\beta$]:<br>
    <br>$$gCache[s] = ('\geq', v) \land \alpha < v < \beta \implies evaluate(s, player\_num, playing\_player, limit, f, \alpha, \beta) = f(s, player\_num, playing\_player, limit, v, \beta)$$<br>
    
    C. the value is less or equal than $\alpha$. The value doesn't help and so the the interval is left at [$\alpha$,$\beta$]:<br>
    <br>$$gCache[s] = ('\geq', v) \land v \leq \alpha \implies evaluate(s, player\_num, playing\_player, limit, f, \alpha, \beta) = f(s, player\_num, playing\_player, limit, \alpha, \beta)$$<br>



In [32]:
gCache = {}
def evaluate(state, player_num, playing_player, limit, f, alpha=-1, beta=1):
    
    global gCache
    state_tuple = to_tuple(state)
    if (state_tuple, player_num, playing_player, limit) in gCache:
        flag, v = gCache[(state_tuple, player_num, playing_player, limit)]
        if flag == '=':
            return v
        if flag == '≤':
            if v <= alpha:
                return v
            elif v < beta: # alpha < v
                w = f(state, player_num, playing_player, limit, alpha, v)
                store_cache(state_tuple, player_num, playing_player, limit, alpha, v, w)
                return w
            else: # beta <= v
                w = f(state, player_num, playing_player, limit, alpha, beta)
                store_cache(state_tuple, player_num, playing_player, limit, alpha, beta, w)
                return w
        if flag == '≥':
            if beta <= v:
                return v
            elif alpha < v: # v < beta
                w = f(state, player_num, playing_player, limit, v, beta)
                store_cache(state_tuple, player_num, playing_player, limit, v, beta, w)
                return w
            else: # v <= alpha
                w = f(state, player_num, playing_player, limit, alpha, beta)
                store_cache(state_tuple, player_num, playing_player, limit, alpha, beta, w)
                return w
    else: # no value stored in gCache for state_tuple
        v = f(state, player_num, playing_player, limit, alpha, beta)
        store_cache(state_tuple, player_num, playing_player, limit, alpha, beta, v)
        return v

### Function: store_cache
The function *store_cache* is an auxiliary function that takes all parameters from evaluate and stores them in the global Cache which is part of the Memoization process.

In [33]:
def store_cache(state, player_num, playing_player, limit, alpha, beta, v):
    global gCache
    if   v <= alpha:
        gCache[(state, player_num, playing_player, limit)] = ('≤', v)
    elif v <  beta: # alpha < v
        gCache[(state, player_num, playing_player, limit)] = ('=', v)
    else: # beta <= v
        gCache[(state, player_num, playing_player, limit)] = ('≥', v)

### Functions: maxValue and minValue
The functions *maxValue* and *minValue* take six arguments:
- *state* is the state of the game,
- *player_num* is either 0 or 1 depending on whose perspective is being evaluated
- *playing_player* is either 0 or 1 depending on whose turn it currently is
- *limit* is an integer to define the recursion depth limit
- *alpha* is the lower bound (further referred to as $\alpha$)
- *beta* is the upper bound (further referred to as $\beta$)

When the game reaches a finished state the utility is returned. If the limits reaches 0 the heurisitc is returned. Then the states following the current state are iterated. The evaluate function is called for each state. Depending on max- or minValue the value resulting from evaluate is compared to $\alpha$ or $\beta$. If value falls below alpha or exceeds beta the value is returned. Otherwise the maximum of $\alpha$ and value or the minimum of $\beta$ and value is returned.      

The functions *minValue* and *maxValue* have three actions but the first two actions are similar for both functions:

1. When the given state s is a terminal state or the depth limit is zero, the Minimax value function is called to calculate the utility (if s $\in$ TerminalStates) or heuristic (if limit is zero) of s: <br>

    $$finished(s) \lor (limit = 0) \implies minValue(state, player\_num, playing\_player, limit, \alpha, \beta) = value(s, playing\_player,  limit)$$

    The same signature is true for *maxValue*.
<br>

2. The following states are stored in a heap to perform move ordering. This means that the successor states are sorted based on the existing information in gCache. If a tuple of a successor state and the fitting player_num, playing_player and limit is already in the cache this gives information on the value of this successor state. This information is used to sort the successor states in the PrioQueue **NS_PrioQueue** with using the flag and the cached values as the ordering of the queue. If a successor state is not cached already, the tuple ('=', 0) is used as a default. By using this method, the successor states which are most likely to bring the most value for the current player are evaluated first. This could prevent further unnessecary evaluations of worse successor states. Move ordering only works because of the progressive deepening which is performed by the function pd_evaluate which is explained further in the next section.
<br>
  
3. As long as there are successor states in **NS_PrioQueue** left, the next successor state ns is extracted from **NS_PrioQueue** and the value v as result of calling *evaluate* with ns and *minValue* (in case of *maxValue*) or *maxValue* (in case of *minValue*) is calculated. Then the following rules apply:

    **A. In case of *maxValue*:** <br>
    $$ (\exists ns \in NS\_PrioQueue: v \ge \beta) \implies maxValue(s, player\_num, playing\_player, limit, \alpha, \beta) = v $$<br>
    $$ (\forall ns \in NS\_PrioQueue: v \lt \beta) \implies maxValue(s, player\_num, playing\_player, limit, \alpha, \beta) = \alpha $$
    Between each iteration, $\alpha$ is updated to the higher value of the previous $\alpha$ and the new v.

    **B. In case of *minValue*:** <br>
    $$ (\exists ns \in NS\_PrioQueue: v \le \alpha) \implies maxValue(s, player\_num, playing\_player, limit, \alpha, \beta) = v $$<br>
    $$ (\forall ns \in NS\_PrioQueue: v \gt \alpha) \implies maxValue(s, player\_num, playing\_player, limit, \alpha, \beta) = \beta $$
    <br> Between each iteration, $\beta$ is updated to the lower value of the previous $\beta$ and the new v.


In [34]:
def maxValue(state, player_num, playing_player, limit, alpha, beta):
    if finished(state) or limit==0:
        return value(state,playing_player,limit)
    
    NS_PrioQueue = []
    for ns,_ in next_states(state, player_num):
        heapq.heappush(NS_PrioQueue, (-gCache.get((to_tuple(ns),other_player(player_num),playing_player,limit-2),('=',0))[1], ns))

    while NS_PrioQueue != []:
        _, ns = heapq.heappop(NS_PrioQueue)
        v = evaluate(ns, other_player(player_num), playing_player, limit-1, minValue, alpha, beta)
        if v >= beta:
            return v
        alpha = max(alpha, v)
    return alpha

In [35]:
def minValue(state, player_num, playing_player, limit, alpha, beta):
    if finished(state) or limit==0:
        return value(state,playing_player,limit)
    
    NS_PrioQueue = []
    for ns,_ in next_states(state, player_num):
        heapq.heappush(NS_PrioQueue, (-gCache.get((to_tuple(ns),other_player(player_num),playing_player,limit-2),('=',0))[1], ns))

    while NS_PrioQueue != []:
        _, ns = heapq.heappop(NS_PrioQueue)
        v = evaluate(ns, other_player(player_num), playing_player, limit-1, maxValue, alpha, beta)
        if v <= alpha:
            return v
        beta = min(beta, v)
    return beta

### Function: pd_evaluate

The function *pd_evaluate* is responsible for performing the progressive deepening of the AlphaBeta algorithm. The main idea behind progressive deepening is to not only evaluate the value of a state in the given limit, but to iteratively increase the limit and evaluate the value for each of these iterations. At first sight this seems to meen a huge increase of node inspections and total calculations. This should not be the case though, because of the memoization due to the function *evaluate* and the cache *gCache*. Instead, the node inspections can be decreased because the evaluation of each limit can use the results from the previous limit calculations by pre-sorting the successor states (performing move ordering) as explained previously.

In [ ]:
def pd_evaluate(state, player_num, playing_player, limit, f, alpha=-1, beta=1):
    global gCache
    for l in range(limit+1):
        value = evaluate(state, player_num, playing_player, l, f, alpha, beta)
        if value in [-1, 1]:
             return value
    return value

### Method: choose_house
The method *choose_house(current_state)* functions similarly to the other AI Player Classes.

For AlphaBeta the only difference is that the function *evaluate* is called to determine the values of a state and therefore make a decision based on that.


In [36]:
def choose_house(self, current_state):
    
    NS = next_states(current_state, self.number)

    bestVal =  pd_evaluate(current_state, self.number, self.number, self.limit, maxValue, -1, 1)

    BestChoices = [choices[0] for next_state,choices in NS if pd_evaluate(next_state, other_player(self.number), self.number, self.limit-1, minValue, -1, 1) == bestVal]

    if len(BestChoices) == 0:
        raise ValueError(f'No choice for AlphaBeta found! \nbestVal: {bestVal}\nBestChoices: {BestChoices}\nnext_states: {NS}')#

    # Find best choice, where number of seeds in own Store is maximized (out of next_states)
    best_choice = rn.choice(BestChoices)

    return best_choice

AlphaBeta.choose_house = choose_house

# Scout Player Class

The class <tt>Scout</tt> inherits from <tt>Player</tt>. It is an implementation of an AI Player that makes game decisions based on the [Scout algorithm](https://www.aaai.org/Papers/AAAI/1980/AAAI80-041.pdf). Scout calculates the best possible outcome for the Player and makes the best decision regarding the current state of the game.

The Scout algorithm consists of the core function EVAL and the auxiliary function TEST which are explained in more detail in the following section. Like the AlphaBeta implementation, the implementation of the Scout algorithm is depth limited because it would take far to long to calculate every game path to it's terminal state.

In [37]:
class Scout(Player):
    
    def __init__(self, name, limit, seed=0):
        self.player_type = "Scout"
        self.limit = limit
        self.seed = seed
        rn.seed(seed)
        super().__init__(name)

### Function: TEST
The function *TEST(state, v, player_num, playing_player, limit, op)* takes six arguments:
- *state* is the state of the game,
- *v* is the given reference value,
- *player_num* is either 0 or 1 depending on whose perspective is being evaluated
- *playing_player* is either 0 or 1 depending on whose turn it currently is
- *limit* is an integer to define the recursion depth limit
- *op* is a function used to compare the value *v* with the computed value of the given *state*
    - op can either be a *greaterThan* comparison or a *greaterOrEquals* comparison

TEST is an auxiliary function for the main Scout algorithm function EVAL. TEST has the purpose to check if the given state satisfies the inequality **value(state) > v** (or ≥, depending on the given mathmatical operator function *op*) for a defined recursion depth *limit*. As a result, TEST returns either **True** or **False**. 

TEST is a recursive function which works as follows:

1. If *state* $\in$ TerminalStates or the maximal recursion depth is reached, **value(state)** is calculated and the inequality **op(value(state), v)** is evaluated.<br><br>

2. Otherwise, the list of successor states **NS** from *state* is calculated with the function *next_states*. Afterwards, the function TEST checks, whether the state value should be maximized or minized in this invocation.<br></br>

    **A. player_num = playing_player**: <br>
    The value should be maximized. TEST iterates through **NS** so that the following rules apply:<br>
        
    $$ \exists s_{n} \in NS: TEST(s_{n}, v, other\_player(player\_num), playing\_player, limit-1, op)) $$<br>$$ \implies TEST(state, v, player\_num, playing\_player, limit, op) := True $$ **(I.)**
    
    $$ (\forall s_{n} \in NS: \neg TEST(s_{n}, v, other_player(player_num), playing_player, limit-1, op)) $$<br>$$ \implies TEST(state, v, player_num, playing_player, limit, op) := False $$ **(II.)**


    **B. $\neg$ (player_num = playing_player)**: <br>
    The value should be minimized. TEST iterates through **NS** so that the following rules apply:<br>

    $$ \exists s_{n} \in NS: \neg TEST(s_{n}, v, other\_player(player\_num), playing\_player, limit-1, op)) $$<br>$$ \implies TEST(state, v, player\_num, playing\_player, limit, op) := False $$ **(I.)**
    
    $$ (\forall s_{n} \in NS: TEST(s_{n}, v, other_player(player_num), playing_player, limit-1, op)) $$<br>$$ \implies TEST(state, v, player_num, playing_player, limit, op) := True $$ **(II.)**
  
It is very important to note that regarding the signatures **A. I.** and **B. I.** only the first successor state for which the revursive call of TEST returns the in the condition specified needed result is relevant. If the condition is fulfilled, the iteration through the list **NS** can be stopped early which means that the number of node inspections is reduced at this point. The advantage from this reduction is especially high if progressive deepening is used for the Scout algorithm as the successor states are presorted in the **NS** list.

#### Function: memoize_test
The function *memoize_test* is an implementation of memoization for the function *TEST*.
Therefore function calls are cached and when the same call is made more than once, the already cached value is returned instead of computing the same result again.

In [38]:
Cache_test = {}

def memoize_test(f):
    global Cache_test

    def f_memoized(*args):
        args = (to_tuple(args[0]),args[1],args[2],args[3],args[4],args[5])
        if args in Cache_test:
            return Cache_test[args]
        
        result = f(to_list(args[0]),args[1],args[2],args[3],args[4],args[5])
        Cache_test[args] = result
        return result

    return f_memoized

In [39]:
def TEST(state, v, player_num, playing_player, limit, op):
   
    # Base Case
    if finished(state) or limit == 0:
        return op(value(state, playing_player, limit), v)

    ns = next_states(state,player_num)
    # if player_num == playing_player then the value of the state should be maximized
    if player_num == playing_player:
        for next_state,_ in ns:
            if TEST(next_state, v, other_player(player_num), playing_player, limit-1, op):
                return True
        return False
    else:
        for next_state,_ in ns:
            if not TEST(next_state, v, other_player(player_num), playing_player, limit-1, op):
                return False
        return True

### Function: EVAL
The function *EVAL(state, player_num, playing_player, limit)* takes four arguments:
- *state* is the state of the game,
- *player_num* is either 0 or 1 depending on whose perspective is being evaluated
- *playing_player* is either 0 or 1 depending on whose turn it currently is
- *limit* is an integer to define the recursion depth limit

The function EVAL is the core function of the Scout algorithm. Similar to the Minimax *value* function and the AlphaBeta *evaluate* function, EVAL calculates the best possible value for *playing_player* recursively in the game state *state* and returns it. The function is depth limited by *limit* and therefore the value is only calculated based on states which are desirable in *limit* steps. 

The main advantage of EVAL in comparison to Minimax is that not all successor states are necessarily evaluated but the first successor. For all following successor states, EVAL uses the function TEST to determine if it is worth evaluating this successor node in more detail. This way, many node inspections are left out, especially when the successor states are pre-sorted due to progressive deepening. 

EVAL is a recursive function which works as follows:

1. If *state* $\in$ TerminalStates or the maximal recursion depth is reached, **value(state)** is calculated and returned.<br><br>

2. Otherwise, the list of successor states **NS** from *state* is calculated using the function *next_states*. Afterwards, the first successor from the list is extracted and evaluated by recursevly calling the EVAL function. This value is set as the initial **best_val**.<br><br>

3. In the next step, the EVAL function iterates the list **NS** to possibly improve the value of **best_val** further. Therefore, the following rules are applied:

    For $s_{n} \in$ **NS** with $n \in$ {1, ..., k}, k being the number of states in **NS**, holds:
    
    $$ ((player\_num = playing\_player) \land TEST(s_{n}, best\_val, opponent, playing\_player, limit-1, \gt)) $$ <br>
    $$ \implies best\_val := EVAL(s_{n}, other\_player(player\_num), playing\_player, limit-1) $$
    
    $$ (\neg (player\_num = playing\_player) \land \neg TEST(s_{n}, best\_val, opponent, playing\_player, limit-1, \ge)) $$ <br>
    $$ \implies best\_val := EVAL(s_{n}, other\_player(player\_num), playing\_player, limit-1) $$
    <br><br>
4. After iterating all states from **NS**, EVAL returns the resulting best value **best_val**.

Eventhough in the Scout algorithm some states need to be visited more than one time as they might be inspected by EVAL and TEST, an exact mathematical analysis from the [research paper](https://www.aaai.org/Papers/AAAI/1980/AAAI80-041.pdf) from Judea Pearl shows that Scout still is optimal in it's branching factor, which makes it very efficient as can is demonstrated in the notebook 'AIComparisons'.


#### Function: memoize_eval
The function *memoize_eval* is an implementation of memoization for the function *EVAL*. Therefore function calls are cached and when the same call is made more than once, the already cached value is returned instead of computing the same result again.

In [41]:
Cache_eval = {}

def memoize_eval(f):
    global Cache_eval

    def f_memoized(*args):
        args = (to_tuple(args[0]),args[1],args[2],args[3])
        if args in Cache_eval:
            return Cache_eval[args]
        
        result = f(to_list(args[0]),args[1],args[2],args[3])
        Cache_eval[args] = result
        return result

    return f_memoized

In [42]:
@memoize_eval
def EVAL(state, player_num, playing_player, limit):
  
    # Base Case
    if finished(state) or limit == 0:
        return value(state, playing_player, limit)

    ns = next_states(state,player_num)
    NS_PrioQueue = []

    factor = 1
    if player_num == playing_player:
        factor = -1

    for s,_ in ns:
        heapq.heappush(NS_PrioQueue, (factor * Cache_eval.get((to_tuple(s),other_player(player_num),playing_player,limit-2), 0), s))

    _,S_1 = heapq.heappop(NS_PrioQueue)
    opponent = other_player(player_num)
    best_val = EVAL(S_1, opponent, playing_player, limit-1)

    while NS_PrioQueue != []:
        _,S_k = heapq.heappop(NS_PrioQueue)
        if player_num == playing_player:
            if TEST(S_k, best_val, opponent, playing_player, limit-1, operator.gt):
                new_best_val = EVAL(S_k, opponent, playing_player, limit-1)
                if new_best_val <= best_val:
                    raise Exception("TEST with > not working properly")
                best_val = new_best_val
        else:
            if not TEST(S_k, best_val, opponent, playing_player, limit-1, operator.ge):
                new_best_val = EVAL(S_k, opponent, playing_player, limit-1)
                if new_best_val >= best_val:
                    raise Exception("TEST with >= not working properly")
                best_val = new_best_val
    
    return best_val

### Function: PD_EVAL

TODO!

In [ ]:
def PD_EVAL(state, player_num, playing_player, limit):
    for l in range(limit+1):
        value = EVAL(state, player_num, playing_player, l)
        if value in [-1, 1]:
             return value
    return value

### Method: choose_house
The method *choose_house(current_state)* functions similarly to the other AI Player Classes.

For AlphaBeta the only difference is that the function *EVAL* is called to determine the values of a state and therefore make a decision based on that.


In [43]:
def choose_house(self, current_state):
    
    NS = next_states(current_state, self.number)

    bestVal =  PD_EVAL(current_state, self.number, self.number, self.limit)

    BestChoices = [choices[0] for next_state,choices in NS if PD_EVAL(next_state, other_player(self.number), self.number, self.limit-1) == bestVal]

    if len(BestChoices) == 0:
        raise ValueError(f'No choice for Scout found! \nbestVal: {bestVal}\nBestChoices: {BestChoices}\nnext_states: {NS}')#

    # Find best choice, where number of seeds in own Store is maximized (out of next_states)
    best_choice = rn.choice(BestChoices)

    return best_choice

Scout.choose_house = choose_house



# Kalah_Game Class

#### Attributes:
- *state*
- *players*
- *current_player*
- *display_mode*
- *logged_moves*

#### Methods:

- *\_\_init__(players)*
- *_show_state(state)*
- *show_state()*
- *start()*
- *log_to_file(file_id)*

The class <tt>Kalah_Game</tt> is the core of the Kalah game implementation. It contains all information on the game, including the current game state.

The attribute *state* represents the state of the Kalah board which is defined by the number of seeds laying in each of the player's house and their stores. It is implemented by a nested list which contains a list for the house on each of the two players' sides. The last index of each of the lists is the store of that player. At the start of the game, where the stores are empty and there are six seeds in every house, the implemented representation of the state for example is: [[4,4,4,4,4,4,0], [4,4,4,4,4,4,0]].

The attribute *players* is a list of the two players that play the game. They must be instances of a subclass of the <tt>Player</tt> class. The order in which they take turns is determined by their <tt>Player</tt> *number*: The player with the number 0 goes first.

The attribute *current_player* has either the value 0 or 1. It takes track of which player's turn it currently is. If *current_player* has the value 0 for example, it is the turn of the player which stands at index 0 of the *players* list and therefore also has the <tt>Player</tt>'s class attribute *number* of value 0.

The attribute *display_mode* defines which way the game is displayed in the output console. The possible values are 0, 1 and 2 and are defined as follows:
- 0: No output is displayed. The game is only logged to the log file
- 1: The board is only displayed using the print method *_show_state*
- 2: The board is displayed with the function *draw_board* using **ipycanvas** for rendering

The attribute *logged_moves* is a list containing a tuple for every move that is made in the game. This list is necessary for the method *log_to_file*. Each tuple has the player number of the player that have made the move at the first index, the index of the chosen house at the second index and the resulting game state as the third index. The start state is saved as the first move in the list, having -1 as the player number and -1 as the chosen house number. As Python uses references, the states are saved in this list by creating deep copies with the library **copy**. At the beginning of the game *logged_moves* looks like this:

\[(-1, -1, [[4,4,4,4,4,4,0], [4,4,4,4,4,4,0]])\]

### Method: __init__
The method *\_\_init__(players)* initializes the game by setting the *state* to the initial state (seen above), initializing a board object and setting the *players* list using the received "players" argument. Before setting the *players* list, the received list is checked for the number of items it contains (must be exactly 2) and if the items in the list are both instances of a <tt>Player</tt> subclass (but not of the class <tt>Player</tt> itself). In error case, matching error messages are raised.

In [44]:
class Kalah_Game():
    
    def __init__(self, players, display_mode):
        self.state = copy.deepcopy(gStartState)
        self.turn = 0
        
        if len(players) != 2:
            raise ValueError("There must be exactly two players!")
        if not ((isinstance(players[0], Player) and type(players[0]) != Player) 
            and (isinstance(players[1], Player) and type(players[1]) != Player)):
            raise ValueError("Both players must be of instances of a subclass of the class Player!")
        
        players[0].number = 0
        players[1].number = 1
        
        self.players = players
        self.current_player = 0
        
        if display_mode not in range(3):
            raise ValueError("The display mode must be 0, 1 or 2!")
        self.display_mode = display_mode
        
        self.logged_moves = [(-1,-1,copy.deepcopy(self.state))]

### Methods: show_state
The private method *show_state(state)* creates a formatted string which represents the received state and prints it to the console. It can be used as an alternative to the **ipycanvas** game UI.

In [45]:
def show_state(self):
    s = f''

    s += f'{self.players[0].name}:\t\t\t'
    for j in range(6,-1,-1):
        s += f'{gOrder[0][j]}: {self.state[0][j]}  '
    s += f'\n'

    s += f'{self.players[1].name}:\t\t\t\t'
    for j in range(7):
        s += f'{gOrder[1][j]}: {self.state[1][j]}  '
    s += f'\n'

    print(s)
    
Kalah_Game.show_state = show_state

### Method: start
The method *start()* starts the Kalah game. Until the game is finished (the method *_finished()* returns True), both players take turns, starting with the player with the number 0. At the start of each turn, the current game state is shown. Next, the current player chooses one of the house with the <tt>Player</tt> method *choose_house(current_state)*. Afterwards, this choice is handed to the private method *_move(player_num, choice)* which calculates the new game state. The attribute *state* is updated with this new game state. Then it is the turn of the other player. If the game is finished, the method *utility()* calculates which of the players wins the game and the result is printed to the console.

In [46]:
def start(self):
    while(not finished(self.state)):
        self.turn += 1
        if self.display_mode == 1:
            print("\nCurrent state:")
            self.show_state()
            print(f"Next is {self.players[self.current_player].name}'s turn.")
        elif self.display_mode == 2:
            print("Current state:")
            draw_board(self.state,self.turn)
            print(f"Next is {self.players[self.current_player].name}'s turn.")
            time.sleep(0.05)
        
        choice = self.players[self.current_player].choose_house(self.state)
        
        if self.display_mode != 0:
            print(f"{self.players[self.current_player].name} chose {gOrder[self.current_player][choice]}")

        self.state, another_turn = move(self.state, self.current_player, choice)
        # create move log
        move_log = tuple([self.current_player, choice, copy.deepcopy(self.state)])
        self.logged_moves.append(move_log)
        
        if not another_turn:
            self.current_player = other_player(self.current_player)
        elif another_turn and self.display_mode != 0:
            print(f"{self.players[self.current_player].name} gets another turn!")

    won0 = utility(self.state,0)
    
    if self.display_mode != 0:
        if self.display_mode == 1:
            print("")
            self.show_state()
        elif self.display_mode == 2:
            draw_board(self.state,self.turn)
            
        print("Finished Game!")  
        
        if won0 == 1:
            print(f"{self.players[0]} wins with {sum(self.state[0])} to {sum(self.state[1])} points!")
        elif won0 == -1:
            print(f"{self.players[1]} wins with {sum(self.state[1])} to {sum(self.state[0])} points!")
        else:
            print(f"Draw!")
    
Kalah_Game.start = start

### Method: log_to_file
The method *log_to_file* creates a json file from all important game information, including the player subclass types and names as well as the different moves from the game and the winner(s). With the use of this method, played games can be saved and analyzed. The created log file is saved to the folder "logs" with the given file_id in the file name.

In [47]:
import json

def log_to_file(self, file_id):
    json_dict = {
        "players":{
            0:{},
            1:{}
        },
        "moves":self.logged_moves,
        "winner":[i for i in range(2) if utility(self.state,i) != -1]
    }
    
    for i in range(2):
        json_dict["players"][i]["type"] = self.players[i].player_type
        json_dict["players"][i]["name"] = self.players[i].name
        if isinstance(self.players[i], Random_AI):
            json_dict["players"][i]["seed"] = self.players[i].seed
        if isinstance(self.players[i], Minimax):
            json_dict["players"][i]["limit"] = self.players[i].limit
    
    with open(f"logs/log_{file_id}.json", "w") as f:
        f.write(json.dumps(json_dict, indent=4))
        f.close()

Kalah_Game.log_to_file = log_to_file

## Repeat Game from Log File

The function *repeat_game* takes the filename of a log file and the display mode with which the game should be repeated as the input values. It also receives the information, if included AIs should calculate their choices new or if they should just repeat the moves that are presented in the log file. The decicions of human players are always repeated based on the log file. 

The function detects the player types and names from the file and creates players based on this information. The player types are only relevant for AIs if **repeat_AIs** is set to False. In all other cases, the players are created from the <tt>Repeated_Player</tt> class.

In the end, the game is initialized with the created players and the given display mode and then started.

In [48]:
def repeat_game(filename, repeat_AIs, display_mode):
    json_dict = {}
    
    with open("logs/"+filename) as f:
        json_dict = json.load(f)
    
    players = []
    for i in range(2):
        i = str(i)
        p_type = json_dict["players"][i]["type"]
        
        if repeat_AIs:
            players.append(Repeated_Player(json_dict["players"][i]["name"],json_dict["moves"]))
        else:
            if p_type == "Human":
                players.append(Repeated_Player(json_dict["players"][i]["name"],json_dict["moves"]))
            elif p_type == "Random_AI":
                players.append(Random_AI(json_dict["players"][i]["name"],json_dict["players"][i]["seed"]))
            elif p_type == "Minimax":
                players.append(Minimax(json_dict["players"][i]["name"],json_dict["players"][i]["limit"]))
            else:
                raise ValueError("The given player type is unknown!")
    
    game = Kalah_Game([players[0],players[1]],display_mode)
    game.start()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b91c3ea7-d814-439b-837a-72fdc90697b1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>